In [ ]:
%run ./required_fxns.py

In [ ]:
list(df.columns)

In [ ]:
df_ = df.copy()
df_ = df_.drop('index', axis=1)
df_.to_csv('df_new.csv', index=False)

In [ ]:
adj = create_adjacency_matrix(df)
G = nx.from_numpy_matrix(adj)

In [ ]:
edges = {}; edges2 = {}
for e in G.edges():
    edges[e[0],e[1]] = (df['email_address'][e[0]], df['email_address'][e[1]])
    edges2[e[0],e[1]] = (df['fuzzy_device_id'][e[0]][:7], df['fuzzy_device_id'][e[1]][:7])

In [ ]:
edges

In [ ]:
[e for e in G.edges]

In [ ]:
df = testdf
mapping = {x:node_id for node_id,x in enumerate(set(df['email'].values.tolist() + df['customer_id'].values.tolist()))}
df['from'] = df['customer_id'].apply(lambda x: mapping[x])
df['to'] = df['email'].apply(lambda x: mapping[x])

In [ ]:
G = nx.from_edgelist(df[['from', 'to']].values, create_using=nx.Graph())

In [ ]:
G.edges

In [ ]:
edges = dict(zip([e for e in G.edges if e[0] != e[1]],
         testdf['email'][~pd.Series(testdf['email']).duplicated(keep='first')]))

In [ ]:
ssh = {}
for x in df[['email', 'from', 'to']].iterrows():
#     print (x[1]['from'], x[1]['to'])
    ssh[x[1]['from'], x[1]['to']] = x[1]['email']
#     print( int(x['from']), int(x['to'])):x['email']
ssh

In [ ]:
import matplotlib
spring_pos = nx.spring_layout(G)
cmap = matplotlib.colors.ListedColormap(['lightgray', 'red'])
# node_sizes = [100 if x>1 else 10 for x in df_out.velocity]
node_colors = testdf.fraud
# edge_colors = ['r' if x==1 else 'g' for x in nx.get_edge_attributes(G, 'fraud').values()]
nx.draw_networkx(G, pos=spring_pos, cmap=cmap,
                 node_color=node_colors, width=3, with_labels=True,
                 labels=dict(zip(range(len(testdf)), testdf['email'])))
plt.show()

In [ ]:
def build_graph(df_input, graph_type=nx.Graph()):
    df = df_input.copy()
    mapping = {x:node_id for node_id,x in
               enumerate(set(df['customer_id'].values.tolist() + df['email'].values.tolist()
                            + df['device_id'].values.tolist()))}
    df['from'] = df['customer_id'].apply(lambda x: mapping[x])
    df['to'] = df['email'].apply(lambda x: mapping[x])
    df['to_2'] = (df['device_id'].apply(lambda x: mapping[x]))

    G = nx.from_edgelist(df[['from', 'to']].values, create_using=graph_type)
    nx.set_edge_attributes(G, {(int(x['from']),int(x['to'])):x['email'] for idx, x in df[['from', 'to', 'email']].iterrows()}, 'email')
    nx.set_edge_attributes(G, {(int(x['from']),int(x['to_2'])):x['device_id'] for idx, x in df[['from', 'to_2', 'device_id']].iterrows()}, 'device_id')

    return G, df

In [ ]:
G, df = build_graph(testdf)
df

In [ ]:
nx.draw(G, with_labels=True, labels=dict(zip(range(len(testdf)), testdf['customer_id'])))
plt.show()

In [ ]:
%matplotlib inline

In [ ]:
import sys
sys.path.append('GRAPH_INSIGHTS/lib')
sys.path

In [ ]:
import networkx as nx
from networkx.drawing.nx_agraph import graphviz_layout

In [ ]:
import os
os.environ['PATH']

In [ ]:
os.environ['PATH'] += os.pathsep+'/root/GRAPH_INSIGHTS/lib/python3.7/site_packages/graphviz/'

In [ ]:
graph_email_network(G, df, edges)

In [ ]:
graph_device_id_network(G, df, edges2)

In [ ]:
search_by_email()

In [ ]:
search_by_fuzzy_device_id()

In [ ]:
search_by_customer_id()

In [ ]:
search_by_email()

In [ ]:
from graphviz import dot
try:
    import pygraphviz
    from networkx.drawing.nx_agraph import graphviz_layout
except ImportError:
    try:
        import pydot
        from networkx.drawing.nx_pydot import graphviz_layout
    except ImportError:
        raise ImportError("This example needs Graphviz and either "
                              "PyGraphviz or PyDotPlus")
plt.figure(figsize=(6,8))
G = nx.complete_graph(5)
pos=nx.nx_pydot.graphviz_layout(G, 'dot')

nx.draw_networkx_nodes(G,pos,nodelist=G.nodes(),node_size=node_sizes,\
linewidths=0.1,vmin=0,vmax=1,alpha=0.8,\
node_color=[D[n] for n in G.nodes()])

nx.draw_networkx_edges(G,pos,edgelist=G.edges(),width=0.1,\
edge_color="black",alpha=0.6)
plt.axis('off')
plt.tight_layout()
plt.show()